# Handelsblatt data

*Handelsblatt* is a popular German daily newspaper. According to the IVW, Informationsgemeinscahft zur Feststellung der Verbreitung von Werbeträgern (Information Community for the Assessment of the Circulation of Media), it had a circulation of 140612 daily copies in the first quarter of 2021. An appealing feature of Handelsblatt for forecasting economic activity is its focus on the economy.

We purchased Handelsblatt data from Genios, a German provider of business information. The corpus includes **980516** articles from January 1994 to November 2018. The data was purchased in February 2019. 

The data set includes 25 subfolders corresponding to a particular year (e.g., HB_1994). Each subfolder contains a few XML files which we parse to extract information relevant to our research project. Unfortunately, the copyright prevents us from publishing the data.

## Load the data

First, we need to read in the data. We create the list including the names of the 25 subfolders (`folder_list`) and apply the function `hb_load` to them in parallel by exploiting Python's `multiprocessing` library.

We extract the following XML elements:

* datum - publication date
* worte - the word count
* ressort - section/subsection of the newspaper
* titel-liste/serientitel - the name of the series
* quelle/name - the name of the newspaper
* titel-liste/titel - article's title
* titel-liste/dachzeile - article's kicker
* titel-liste/untertitel - article's subheading
* inhalt/vorspann - annotation
* inhalt/text - text of the article

In [1]:
import os

# Handelsblatt is the main folder with 25 subfolders in it
path = 'E:\\Userhome\\mokuneva\\Handelsblatt' # your path here

# Create the list of all subfolders within Hb main folder.
folder_list=[]

for f in [f for f in os.listdir(path) ]: 
    # os.listdir(path) - names of directories                                         
    folder_list.append(path + '\\' + f)

In [2]:
import multiprocessing as mp # use multiprocessing module for parallel computing

NUM_CORE = mp.cpu_count()-4 # set the number of cores to use

print("The number of cores that will be used: {}".format(NUM_CORE))

The number of cores that will be used: 12


In [3]:
from datetime import datetime
startTime = datetime.now() # track time

import pandas as pd # load pandas: python data analysis library
import hb_load # import a function that loads the data from one folder (see hb_load.py file for details)

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    data_intermediate = pool.map(hb_load.hb_load, folder_list) # load data from each folder in parallel
    data = pd.concat(data_intermediate) # concatenate DataFrames corresponding to different folders
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:01:16.685713


In [4]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [5]:
data[10:15]

,day,kicker,month,newspaper,rubrics,series_title,texts,title,title_only,word_c,year
10,3,,1,Handelsblatt,Nachrichten,,Nachrichten. Yuan abgewertet dpa PEKING. China...,Nachrichten.,Nachrichten.,68,1994
11,3,,1,Handelsblatt,Nachrichten,,Nachrichten. Ruecktritt abgelehnt afp NEU DELH...,Nachrichten.,Nachrichten.,58,1994
12,3,,1,Handelsblatt,Nachrichten,,Nachrichten. Ungarn wertet Forint ab rtr BUDAP...,Nachrichten.,Nachrichten.,33,1994
13,3,,1,Handelsblatt,Nachrichten,,Nachrichten. FDP zum Beamtenrecht dpa BONN. Di...,Nachrichten.,Nachrichten.,55,1994
14,3,,1,Handelsblatt,Nachrichten,,Nachrichten. Neue Drohungen dpa HAMBURG. Der F...,Nachrichten.,Nachrichten.,86,1994


In [6]:
# the number of article before pre-processing
len(data)

980516

## Light pre-processing

### Remove short articles (<100 words)

The first part of the article 327123 contains different symbols (e.g. zw|lf) instead of umlauts. The second part (which I keep) has the same content with the right format. The second part starts with 'NEU DELHI.'.

In [7]:
data.at[327123, 'texts'] = data['texts'][327123][data['texts'][327123].find('NEU DELHI.'):]

An important feature of the text is its length. There are a few ways to count the number of words in a text:

1. Use the metadata ('worte') of an XML file. Beware that numbers are counted as words. This is the column 'word_c' in the DataFrame. 
2. Quick and dirty approach: split the tokens by space and calculate the length of the list. Numbers and other non-alphabetic characters are counted as words. If a space is used as a delimiter in large numbers (100 000), this number will be counted as two tokens.
3. Use `count_words_mp` function, which counts only words. This is our preferred method because we exclude numbers from the analysis in both sentiment analysis and topic modeling.

In [8]:
# the second approach
# data['w_count'] = data['texts'].str.split(' ').str.len()

In [8]:
from datetime import datetime
startTime = datetime.now() # track time

import count_words_mp # import the function calculating the number of words in a text

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:38.511450


In [9]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

The difference between the first ('word_c' column) and third ('word_count' column) approaches might be significant. 

In [10]:
diff = (data['word_c']-data['word_count'])

In [11]:
sorted(diff, reverse=True)[:10]

[695, 691, 591, 430, 429, 429, 404, 390, 387, 387]

This happens because the third approach does not count a number as a word.

In [12]:
data['texts'][list(diff).index(387)][:300]

'Fußball-Bundesliga. \\\\\\\\\\\\\\ zu Hause\\ auswärts\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Diff.\\ Pkt.\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Pkt.\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Pkt.\\ 1. ( 1)\\ Bayern München 34\\ 20\\ 11\\ 3\\ 68:34\\ +34\\ 71\\ 17\\ 13\\ 4\\ 0\\ 37:12\\ 43\\ 17\\ 7\\ 7\\ 3\\ 31:22\\ 28\\ 2. ( 2)\\ Bayer Leverkusen 34\\ 21\\ 6\\ 7\\ 69:41\\ +28\\ 69'

Because short texts have sparse semantic features, topic models and BOW-based sentiment tools perform better on longer texts. This is why we keep the texts longer than 100 words.

In [13]:
# remove articles with the number of words<100
data = data[data['word_count']>=100]

In [14]:
# the number of articles after removing short articles
len(data)

673430

### Remove exact duplicates

A few examples of duplicates in our corpus:
* The same article enters the corpus twice with different publication dates (e.g., 6.4.1994 and 27.4.1994). In this case, a natural solution is to keep the first entry.
* The same article appears twice with a slight variation in the metadata (e.g., one of the duplicates includes the title of the series, or the word count is a little different even though the articles are identical).
* The same article enters the corpus twice with the same publication date and metadata.

In [15]:
# All the duplicated articles are saved as 'hb_duplicates' for further exploration.
hb_duplicates = data[data['texts'].duplicated(keep = False)]

In [16]:
hb_duplicates[:10]

,day,kicker,month,newspaper,rubrics,series_title,texts,title,title_only,word_c,year,word_count
18404,29,,4,Handelsblatt,Weltwirtschaft,,Importlizenzen im Vergleichsverfahren. HB DUES...,Importlizenzen im Vergleichsverfahren.,Importlizenzen im Vergleichsverfahren.,121,1994,112
18618,2,,5,Handelsblatt,Weltwirtschaft,,Importlizenzen im Vergleichsverfahren. HB DUES...,Importlizenzen im Vergleichsverfahren.,Importlizenzen im Vergleichsverfahren.,122,1994,112
29129,7,,7,Handelsblatt,Finanzzeitung; Geld und Kredit,,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,638,1994,604
29637,8,,7,Handelsblatt,Finanzzeitung; Geld und Kredit,,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,636,1994,604
32437,28,,7,Handelsblatt,Wirtschaft und Politik,,Glossar. Das Verarbeitende Gewerbe ist in West...,Glossar.,Glossar.,139,1994,130
35680,19,,8,Handelsblatt,Karriere,,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,176,1994,169
35962,19,,8,Handelsblatt,Karriere,,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,176,1994,169
40771,20,,9,Handelsblatt,Unternehmen und Maerkte,,Hochtief will seine Beteiligung bei Holzmann e...,Hochtief will seine Beteiligung bei Holzmann e...,Hochtief will seine Beteiligung bei Holzmann e...,738,1994,715
40776,20,,9,Handelsblatt,Unternehmen und Maerkte,,Hochtief will seine Beteiligung bei Holzmann e...,Hochtief will seine Beteiligung bei Holzmann e...,Hochtief will seine Beteiligung bei Holzmann e...,738,1994,715
41355,22,,9,Handelsblatt,Wirtschaft und Politik; Konjunkturbarometer,,Glossar. Das Verarbeitende Gewerbe ist in West...,Glossar.,Glossar.,133,1994,126


In [17]:
# drop the exact duplicates, keep the article with the earlier publication date ('first')
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [18]:
# the number of articles after removing exact duplicates
len(data)

670682

## Filtering

### Section

Handelsblatt's articles are organized into 2073 unique sections/subsections. We investigate the most frequently met ones and remove artiles published within a few of them.

In [19]:
# Sections and the number of articles per section
import collections
counter_sections = collections.Counter(data.rubrics)
print(counter_sections.most_common(10))

[('Unternehmen und Märkte', 87617), ('Wirtschaft und Politik', 33223), ('Deutschland', 29652), ('Finanzzeitung', 28197), ('Unternehmen & Märkte', 19289), ('Meinung und Analyse', 17238), ('Titelseite', 16957), ('Beilage oder Sonderseite', 16278), ('International', 16252), ('Europa', 15709)]


To ensure consistency in topical content over time, we remove articles from a few sections that were covered only within a limited time period:

* 1) 'Karriere': news related to work, career; due to the organizational changes, starting from 2014, this section is only present in a weekend edition of the newspaper.
* 2) 'Weekend Journal': the section was introduced in 2002 and did not receive any coverage after 2008. Moreover, the news discussed within this section is on average longer (663 words) than the news from the main section of interest "Wirtschaft und Politik" (436 words).
* 3) 'Panorama': news from around the world, issued in 1997, existed until 2001.
* 4) 'Business-Travel': travel-related news, e.g. weather, 2001-2006, only 366 articles in total.
* 5) 'Fortschritt': scientific findings, new devices, 2001-2003.
* 6) 'Wochenende': weekend news, 2012-2018, average article length is 1579.
* 7) 'perspektiven': perspectives, non-economic topics like a prize for women in science, working abroad, or new beauty standards popularized by Dove's advertising campaign; 2007-2009.

In [20]:
abandoned_sections = ['Karriere', 'Weekend Journal', 'Panorama', 'Business-Travel', 
                    'Fortschritt', 'Wochenende', 'perspektiven']

In [21]:
data = data[~data['rubrics'].isin(abandoned_sections)]

In [22]:
# the number of articles after removing articles from the sections covered only within a certain time period
len(data)

658481

Here we want to exclude the articles from non-economic sections. The list of irrelevant sections follows.

* 1) 'Sportreport': sport news.
* 2) 'Kunstmarkt': art market.
* 3) 'Sportreport      Nachrichten': sport news.
* 4) 'Neue Bücher': an advertisement for new books.
* 5) 'Wissenschaft & Debatte': science and debate.
* 6) 'Literatur': literature.
* 7) 'Leserbriefe': letters to the editor; starting from 2013, there are almost no letters/comments from the readers, because all the comments became digital.
* 8) 'Reisen und Tagen': travel.
* 9) 'Kultur': culture.
* 10) 'Termin- und Optionsmärkte': futures and options markets, mostly quantitative info.
* 11) 'Galerie des Stils': style and fashion.
* 12) 'Galerie': non-economic news about a pop band, Nutella restaurant etc.
* 13) 'Neue Buecher': new books.
* 14) 'Online': news about the websites, digitalization, online services.
* 15) 'Forschung und Technik': research and technology.
* 16) 'Business-Service': news unrelated to the economy, e.g. telephone service for weather forecast, travel information etc.
* 17) 'Computer und Kommunikation': computers and communication.
* 18) 'Ökonomie & Bildung': economics and education, Handelsblatt's materials for school lessons, MBA studies, educating articles about how the economy works.
* 19) 'Auto-Mobil': new car models.
* 20) 'Die Handelsblatt-Woche': announcement of events taking place this week.
* 21) 'Wirtschaftsbuch': economics books.

In [23]:
non_economic_sections = ['Kunstmarkt', 'Sportreport', 'Sportreport      Nachrichten', 
                  'Neue Bücher', 'Wissenschaft & Debatte',  
                  'Literatur', 'Leserbriefe', 'Reisen und Tagen', 
                  'Kultur',  'Termin- und Optionsmärkte',
                  'Galerie des Stils', 'Galerie', 'Neue Buecher', 'Online', 'Forschung und Technik', 
                  'Business-Service', 'Computer und Kommunikation', 'Ökonomie & Bildung', 'Auto-Mobil', 
                  'Die Handelsblatt-Woche', 'Wirtschaftsbuch']

In [24]:
data = data[~data['rubrics'].isin(non_economic_sections)]

In [25]:
# the number of articles after removing articles from the non-economic sections
len(data)

621317

Additionally, we remove articles from the related sections and subsections. 

* This means that we not only remove articles from a section 'Kunstmarkt', but also from the subsections 'Kunstmarkt      Anleihen', 'Kunstmarkt      Aus den Galerien' etc. 

* We consider 'Kunstmarkt' and 'Themen und Trends      Kunstmarkt' as two close sections. 

* The full list of sections/subsections we remove can be found in 'subsections_hb.txt'.

In [26]:
# this function loads the dictionary with the related sections and subsections we want to exclude
from os import path
import codecs

def dictionary_open(name):
    with codecs.open(path.join(os.getcwd(), name),
               'r',  'utf-8') as f:
          dictionary = set(f.read().splitlines()[1:-1])
    return dictionary

In [27]:
subsections_clean = dictionary_open('subsections_hb.txt')

In [28]:
data = data[~data['rubrics'].isin(subsections_clean)]
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [29]:
# the number of articles after removing articles from the related subsections
len(data)

588837

In [30]:
# Sections and the number of articles per section
counter_sections=collections.Counter(data['rubrics'])
print(counter_sections.most_common(10))

[('Unternehmen und Märkte', 87617), ('Wirtschaft und Politik', 33223), ('Deutschland', 29652), ('Finanzzeitung', 28197), ('Unternehmen & Märkte', 19289), ('Meinung und Analyse', 17238), ('Titelseite', 16957), ('Beilage oder Sonderseite', 16278), ('International', 16252), ('Europa', 15709)]


### Section + text

Remove articles for which the following two conditions are met: a section is '' (unclassified), and a text contains a string 'KARRIERE'. As discussed, this type of articles is removed due to organizational changes (see above).

In [31]:
data = data[~((data['rubrics']=='') & (data['texts'].str.contains('KARRIERE')))]

Remove articles that are characterized by the following conditions: a section contains a string "Recht und Steuern" (Law and taxes), and a text contains one of the legal terms from the list `legal_terms`.

* Az./AZ/Aktenzeichen/AKTENZEICHEN - docket number
* LAG/Landesarbeitsgericht - Regional Labour Court
* BAG/Bundesarbeitsgericht - Federal Labour Court
* VI R/IV A - case/law reference
* BFH - Bundesfinanzhof, German Federal Fiscal court
* Abs. - section
* BGH/Bundesgerichtshof - Federal Supreme Court
* BGB - Bürgerliches Gesetzbuch, Civil Code
* BSG - Bundessozialgericht, Federal Social Court
* EStG/Einkommensteuergesetz - Income Tax Act
* OLG/Oberlandesgericht - Higher Regional Court
* Anwaltgerichtshof - Lawyers' Court
* Finanzgericht - Tax Court

We remove these articles because they contain very detailed explanations of laws/tax deduction rules/court decisions which have a low chance to be relevant for forecasting economic activity.

In [32]:
legal_terms = ['Az\\.', 'AZ', 'Az\\:', 'Aktenzeichen', 'AKTENZEICHEN', 'LAG', 'Landesarbeitsgericht', 'Bundesarbeitsgericht', 'BAG', 'Arbeitsgericht', 'VI R','IV A','BFH','Abs\\.','BGH', 'Bundesgerichtshof', 'BGB','BSG','EStG','Einkommensteuergesetz','OLG', 'Oberlandesgericht', 'Anwaltgerichtshof', 'Finanzgericht']
data = data[~((data.rubrics.str.contains('Recht und Steuern')) & (data.texts.str.contains('|'.join(legal_terms))))]

In [33]:
# the number of articles after filtering out articles based on the section and text
len(data)

583767

### Section + title

Remove articles satisfying two conditions: section is 'Inhalt' (content), and the title is not 'Termine des Tages.' Section 'Inhalt' mainly includes news reports about the Handelsblatt and its organization. The aritlces with the title 'Termine des Tages.' may include short news on the Economy.

In [34]:
data = data[~(((data['rubrics']=='Inhalt') | (data['rubrics']=='Inhalt ;') | (data['rubrics']=='Inhalt      Der Werber-Rat'))
              & (data['title']!='Termine des Tages.'))]

In [35]:
# the number of articles after removing articles from the section 'Inhalt'
len(data)

582998

### Title

Exclude articles with the following title patterns:

* book suggestions (Buchtip)
* some very short news on a few people (BUSINESS LOUNGE.)
* articles about Handelsblatt and its organization (Liebe Leser)
* articles with comments from the readers (DIE MEINUNG UNSERER LESER.)
* announcement of events taking place this week (DIE HANDELSBLATT-WOCHE.)
* non-economic news related to numbers (e.g., how many people watched the movie this week); Bilanz des Wochenendes.
* sport news (SPORT TELEGRAMM.)

In [36]:
title_patterns = ['BUCHTIP', 'Buchtip', 'BUSINESS LOUNGE\\.', 'Liebe Leser', 'DIE MEINUNG UNSERER LESER\\.', 
                 'DIE HANDELSBLATT\\-WOCHE\\.', 'Die Handelsblatt\\-Woche\\.', 'Die Handelsblattwoche\\.',
                 'DIE HANDELSBLATTWOCHE\\.', 'Bilanz des Wochenendes\\.', 'BILANZ DES WOCHENENDES\\.',
                 'SPORT TELEGRAMM\\.']
data = data[~data['title_only'].str.contains('|'.join(title_patterns))]

Exclude articles with the following titles:

* Kontakte. (contacts of organizations: addresses and phone numbers).

In [37]:
data = data[~(data['title_only']=='Kontakte.')]

In [38]:
# the number of articles after excluding articles based on the title patterns
len(data)

580713

### Series

Exclude articles from the following non-economic series:

* Neue Wirtschaftsliteratur (Handelsblatt-Beilage): economic books;
* Golf (Handelsblatt-Beilage): golf;
* Literatur (Handelsblatt-Beilage): literature;
* Macher des Handelsblatts (Handelsblatt-Serie): about journalists working for Handelsblatt.

In [39]:
series_exclude = ['Neue Wirtschaftsliteratur (Handelsblatt-Beilage)', 
                  'Golf (Handelsblatt-Beilage)', 'Literatur (Handelsblatt-Beilage)', 
                  'Literatur (Handelsblatt - Beilage)', 'Macher des Handelsblatts (Handelsblatt-Serie)']
data = data[~data['series_title'].isin(series_exclude)]
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [40]:
# the number of articles after excluding articles from the non-economic series
len(data)

580065

In [41]:
counter=collections.Counter(data['year'])
print(counter)

Counter({2000: 33949, 1999: 32861, 2001: 30745, 1996: 27326, 1997: 27085, 1998: 26911, 2004: 26432, 1995: 26170, 1994: 25181, 2002: 25083, 2005: 24726, 2003: 24350, 2006: 24167, 2010: 23853, 2007: 23824, 2008: 23540, 2009: 23354, 2011: 22200, 2012: 20667, 2013: 17507, 2014: 17401, 2015: 16302, 2016: 13111, 2017: 12152, 2018: 11168})


## Umlauts

To fix the issue with umlauts, we use the notebook Umlauts_fix written in Python 2. 

In [42]:
umlauts = ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']
umlauts_replace = ['ae', 'oe', 'ue', 'ss', 'AE', 'OE', 'UE']

In [43]:
hb_umlauts_fix = data[(data.texts.str.contains('|'.join(umlauts_replace))) & (~data.texts.str.contains('|'.join(umlauts))) & (data.year<1999)]

In [44]:
# example of the text that we try to fix with a spellchecker
hb_umlauts_fix.texts[0]

'1994 wird ein Jahr des Wandels. Auf ein Neues!. Steuern und Beitraege steigen, staatliche Leistungen werden gesenkt - wahrlich kein berauschender Auftakt fuer das neue Jahr. Ohnehin sind die Erwartungen nicht sehr rosig. Zwar wird sich die Wirtschaft erholen. Ob es zu einem kraeftigen Aufschwung reicht, ist ungewiss. Sicher ist dagegen der Anstieg der Arbeitslosigkeit. Fuer Spannung sorgt der Wahlmarathon, der erst mit den Bundestagswahlen im Oktober entschieden wird. Kein Zweifel: 1994 wird ein Jahr der Unsicherheit und des Wandels. Ob es auch ein gutes Jahr wird, haengt davon ab, wie dieser Wandel bewaeltigt wird. 1993 war das Jahr der Krisenerkenntnis. Die Einsicht in die Kluft zwischen wirtschaftlich Machbarem und Anspruchsdenken hat erste Ansaetze zur Krisenbewaeltigung ermoeglicht. In der Tarifpolitik und bei den Sozialleistungen hat es - noch vor kurzem undenkbare - Einschnitte gegeben. Auch 1994 wird es Kuerzungen geben. Nur: Mit dem Rotstift allein kann kein Ausweg aus der Kr

In [66]:
hb_umlauts_fix.to_csv('hb_umlauts_fix.csv', encoding='utf-8-sig', sep = ';')

In [45]:
import pandas as pd
hb_umlauts_fixed = pd.read_csv('hb_umlauts_fixed.csv', encoding = 'utf-8', sep=';')

X:\conda\envs\py3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
data.loc[hb_umlauts_fixed['Unnamed: 0.1'], 'texts'] = hb_umlauts_fixed.texts.values

In [47]:
# fixed version
data.texts[0]

'1994 wird ein Jahr des Wandels. Auf ein Neues!. Steuern und Beiträge steigen, staatliche Leistungen werden gesenkt - wahrlich kein berauschender Auftakt für das neue Jahr. Ohnehin sind die Erwartungen nicht sehr rosig. Zwar wird sich die Wirtschaft erholen. Ob es zu einem kräftigen Aufschwung reicht, ist ungewiss. Sicher ist dagegen der Anstieg der Arbeitslosigkeit. Für Spannung sorgt der Wahlmarathon, der erst mit den Bundestagswahlen im Oktober entschieden wird. Kein Zweifel: 1994 wird ein Jahr der Unsicherheit und des Wandels. Ob es auch ein gutes Jahr wird, hängt davon ab, wie dieser Wandel bewältigt wird. 1993 war das Jahr der Krisenerkenntnis. Die Einsicht in die Kluft zwischen wirtschaftlich Machbarem und Anspruchsdenken hat erste Ansätze zur Krisenbewältigung ermöglicht. In der Tarifpolitik und bei den Sozialleistungen hat es - noch vor kurzem undenkbare - Einschnitte gegeben. Auch 1994 wird es Kürzungen geben. Nur: Mit dem Rotstift allein kann kein Ausweg aus der Krise gezeic

## Remove English articles

With the next pre-processing step, we filter out news articles written in English. To do that, we use a **langdetect** library.

In [49]:
startTime = datetime.now()

import identify_eng_2

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    eng_results = pool.map(identify_eng_2.identify_eng_2, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:32:44.658063


In [50]:
data['language'] = eng_results
data = data[data.language==0]
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [51]:
# the number of articles after excluding English articles
len(data)

580006

## Remove URLs

Remove URLs and html file references.
E.g.: 'de/studie99.html', 'Amazon.com', etc.

In [52]:
startTime = datetime.now()

import correct_url

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url.correct_url, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:02:42.346244


In [53]:
data['texts'] = url_corrected

## O instead of 0 problem

In some cases, Optical Character Recognition (OCR) can not distinguish between '0' and 'O' ('o'). As a result, there are tokens like '1OO'. Using regular expressions, we identify problematic tokens and replace 'O' ('o') with '0'.

In [54]:
startTime = datetime.now()

import ocr_replace

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    ocr_corrected = pool.map(ocr_replace.ocr_replace, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:15.561995


In [55]:
data['texts'] = ocr_corrected

## Fixing tokens containing a number and a word

In quite a few cases, a number and a word are erroneously merged together into a single token. Splitting these tokens into two tokens helps us to deal with the following problems:

1. Hyphen-separated words will have the same spelling across the whole corpus. Examples: 20Jährige/20-Jährige, 100prozentig/100-prozentig, 30minütigen/30-minütigen etc.

2. Numbers and the names of the currencies will represent two different tokens. Examples: 100DM/100 DM, 30Euro/30 Euro.

3. Numbers and the measures of time/weight/distance etc. will be split into two tokens. Examples: 100km/100 km, 1Mill/1 Mill, 16Uhr/16 Uhr.

4. Simple mistakes will be corrected: 30bis 40/30 bis 40, 10Fahrzeuge/10 Fahrzeuge, 10und/10 und.

5. Mistakes in the beginnning of the sentences will be corrected: 10Welche/10 Welche, 8Wie/8 Wie, etc.

The most frequently met exceptions from this rule are taken into account:

1. Names of the companies/organizations: 1822direkt, 3Sat, 4MBO etc.
2. Names of smartphone/airplane/satellite models: 4S, 1B, 1k, 328Jet, 5C, 777X etc.
3. German nouns like 90er and English adjectives like 21st, 3rd.

In [56]:
startTime = datetime.now()

import split_number_word

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:24.809659


In [57]:
data['texts'] = split_corrected

## Remove fuzzy duplicates

In [58]:
data.to_csv('before_fuzzy_duplicates.csv', encoding = 'utf-8-sig', sep = ';')

In [ ]:
#data = pd.read_csv('before_fuzzy_duplicates.csv', encoding = 'utf-8-sig', sep = ';')

In [72]:
# Prepare inputs
inputs_year = []
inputs_month = []
inputs_month_year = []
for year in list(set(data['year'])):
    for month in list(set(data['month'])):
        inputs_year.append(year)
        inputs_month.append(month)
        inputs_month_year.append(data[(data['year'] == year) & (data['month'] == month)][["month", "year", "texts"]])
        
#from itertools import repeat
inputs = list(zip(inputs_year, inputs_month, inputs_month_year))

In [75]:
import multiprocessing as mp # use multiprocessing module for parallel computing

NUM_CORE = mp.cpu_count()-4 # set the number of cores to use

In [76]:
import fuzzy_duplicates # import a function that outputs the indices of duplicates 

X:\conda\envs\py3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [77]:
from datetime import datetime
startTime = datetime.now() # track time
delete_indices = []
import pandas as pd # load pandas: python data analysis library
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    # apply function to all combinations of month-year in parallel
    delete_intermediate = pool.map(fuzzy_duplicates.fuzzy_duplicates, inputs)
    delete_indices = delete_indices + delete_intermediate # create one list of indices
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:08:11.215056


In [105]:
len(delete_indices[0])

148

In [92]:
list(set([item for sublist in delete_indices for item in sublist]))

[49154,
 90117,
 57353,
 540686,
 565264,
 540690,
 540691,
 540695,
 540696,
 475180,
 41005,
 32816,
 49,
 41009,
 73778,
 53,
 73782,
 55,
 401466,
 352315,
 475200,
 24641,
 573509,
 81996,
 376913,
 98387,
 8282,
 8283,
 254042,
 65634,
 467044,
 106,
 442475,
 483434,
 49261,
 106608,
 49266,
 565369,
 90235,
 106620,
 442493,
 32905,
 41098,
 352397,
 532624,
 154,
 16540,
 73885,
 164,
 483495,
 204968,
 24745,
 8366,
 336053,
 467126,
 106679,
 82108,
 82109,
 147646,
 8383,
 106687,
 73922,
 82114,
 532675,
 57542,
 65744,
 139477,
 565464,
 222,
 41182,
 225,
 90338,
 368866,
 41189,
 499941,
 516326,
 434410,
 49387,
 90347,
 442611,
 139508,
 33018,
 24833,
 213258,
 221455,
 8465,
 172309,
 221461,
 33049,
 98593,
 106786,
 483621,
 65836,
 221487,
 426288,
 491826,
 307,
 303416,
 573754,
 98620,
 41279,
 82240,
 516422,
 57674,
 57677,
 90452,
 49493,
 24924,
 90463,
 172383,
 205153,
 254313,
 16747,
 196971,
 74093,
 74098,
 123255,
 467321,
 8572,
 311677,
 387,
 659

In [89]:
data['texts'][1743]

'TERMINMÄRKTE / Tagesbericht DTB und Optionshandel. Handel zuversichtlicher. HB FRANKFURT / M. An der Deutschen Terminbörse (DTB) war zum Wochenausklang wieder wachsender Optimismus zu verspüren. Zum Teil schlug sich dies in einer deutlichen Umsatzausweitung nieder. Die Favorisierung der Aktien - Kaufoptionen führte hier zu einer erheblichen Verringerung im Put / Call - Ratio; besonders kräftig war die Belebung im Bereich der Dax - Optionen. Folgende Umsätze wurden gemeldet: 41571 Aktienoptionen, (29947 Calls und 11624 Puts), Put / Call - Ratio: 0,39; 100761 Dax - Optionen, (62808 Calls und 37953 Puts), Put / Call - Ratio: 0,60; 234 Dax - Futures - Optionen (63 Calls und 171 Puts), Put / Call - Ratio: 2,71; 1253 Bund - Futures - Optionen, (473 Calls und 780 Puts), Put / Call - Ratio: 1,65; 0 Bobl - Futures - Optionen; 64960 Bund - Futures; 23269 Bobl - Futures; 24790 Dax - Futures. Die DTB meldet vom Optionshandel auf 16 führende deutsche Standardaktien folgende Kontraktzahlen (zunächs

In [82]:
data['texts'][10159]

'TERMINMÄRKTE / Tagesbericht DTB und Optionshandel. Handel deutlich ruhiger. HB FRANKFURT / M. Nach dem kräftigen Umsatzanstieg der letzten Tage präsentierte sich die Deutsche Terminbörse (DTB) am Donnerstag vor allem in Aktien - und Dax - Optionen wieder ruhiger. Auffallend hoch war dagegen der Handel in Optionen auf Bund - Futures. Gemeldet wurden: 47202 Aktienoptionen, (27906 Calls, 19296 Puts), Put / Call - Ratio (PCR): 0,69; 115169 Dax - Optionen, (62107 Calls, 53062 Puts), PCR: 0,85; 311 Dax - Futures - Optionen (250 Calls, 61 Puts), PCR: 0,24; 1225 Bund - Futures - Optionen1, (583 Calls, 642 Puts), PCR: 1,10; 160 Bobl - Futures - Optionen1, (20 Calls, 140 Puts), PCR: 7,00; 21228 Dax - Futures; 54852 Bund - Futures1; 24547 Bobl - Futures1; 175 Buxl - Futures1; 1504 Fibor - Futures1. Die DTB meldet vom Optionshandel auf 15 führende deutsche Standardaktien folgende Kontraktzahlen (erst Calls, dann Puts): Allianz Holding (505, 505), BASF (941, 203), Bayer (958, 577), Hypo - Bank (30

In [87]:
data[data.texts.str.contains('PFLANZENÖL / ASA - Studie mit optimistischer Prognose.')]

,day,kicker,month,newspaper,rubrics,series_title,texts,title,title_only,word_c,year,word_count,language
49,3,,1,Handelsblatt,Termin- und Rohstoffmaerkte,,PFLANZENÖL / ASA - Studie mit optimistischer P...,PFLANZENOEL / ASA-Studie mit optimistischer Pr...,PFLANZENOEL / ASA-Studie mit optimistischer Pr...,358,1994,345,0


In [115]:
import importlib
importlib.reload(fuzzy_duplicates_test)
#import fuzzy_duplicates_test

fuzzy_duplicates_test.fuzzy_duplicates_test(inputs[0])

X:\conda\envs\py3\lib\site-packages\gensim\matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,doc,duplicate,index_d,index_nd,shorter,similarity
0,PFLANZENÖL / ASA - Studie mit optimistischer P...,PFLANZENÖL / ASA - Studie prognostiziert weite...,49,233,PFLANZENÖL / ASA - Studie mit optimistischer P...,0.967183
1,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,53,154,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.953251
2,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,222,53,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.943061
3,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht von der DTB und vo...,53,307,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.925805
4,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,53,387,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.913084
5,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,458,53,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.946598
6,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,53,545,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.937662
7,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,53,658,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.919347
8,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht von der DTB und vo...,755,53,TERMINMÄRKTE / Tagesbericht von der DTB und vo...,0.936908
9,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,53,921,TERMINMÄRKTE / Tagesbericht DTB und Optionshan...,0.942079


In [114]:
data.iloc[154]

day                                                             5
kicker                                                           
month                                                           1
newspaper                                            Handelsblatt
rubrics                                             Finanzzeitung
series_title                                                     
texts           TERMINMÄRKTE / Tagesbericht DTB und Optionshan...
title           TERMINMAERKTE / Tagesbericht DTB und Optionsha...
title_only      TERMINMAERKTE / Tagesbericht DTB und Optionsha...
word_c                                                        311
year                                                         1994
word_count                                                    231
language                                                        0
Name: 154, dtype: object

In [106]:
delete_indices[0]

[49,
 53,
 222,
 53,
 53,
 458,
 53,
 53,
 755,
 53,
 53,
 53,
 53,
 53,
 1471,
 53,
 1743,
 55,
 106,
 222,
 154,
 154,
 458,
 154,
 658,
 755,
 921,
 1019,
 1101,
 154,
 1391,
 1471,
 154,
 1743,
 164,
 222,
 222,
 222,
 222,
 222,
 222,
 222,
 222,
 222,
 222,
 222,
 1471,
 222,
 1743,
 225,
 307,
 458,
 545,
 658,
 755,
 921,
 1019,
 1101,
 307,
 1391,
 1471,
 307,
 307,
 1743,
 458,
 545,
 755,
 921,
 1019,
 1101,
 387,
 1471,
 387,
 1743,
 404,
 458,
 458,
 755,
 458,
 458,
 458,
 458,
 458,
 1471,
 458,
 1743,
 658,
 755,
 921,
 1019,
 1101,
 545,
 1391,
 1471,
 545,
 1743,
 552,
 755,
 658,
 658,
 658,
 658,
 1391,
 1471,
 658,
 1743,
 755,
 755,
 755,
 755,
 755,
 1471,
 755,
 1743,
 921,
 921,
 921,
 1391,
 1471,
 921,
 1743,
 942,
 1019,
 1019,
 1391,
 1471,
 1019,
 1743,
 1101,
 1391,
 1471,
 1101,
 1101,
 1743,
 1391,
 1471,
 1569,
 1663,
 1743,
 1310,
 1471,
 1391,
 1743,
 1408,
 1471,
 1743,
 1481,
 1743]